<a href="https://colab.research.google.com/github/KUNALDEBNATH/Tango/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
pip install transformers sentence-transformers scikit-learn pandas


In [8]:
import pandas as pd
import numpy as np
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

# Load historical ticket data
df = pd.read_csv('/content/Historical_ticket_data.csv')


# === Agent: Summarizer ===
class SummarizerAgent:
    def __init__(self):
        self.summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

    def summarize(self, text):
        summary = self.summarizer(text, max_length=100, min_length=25, do_sample=False)
        return summary[0]['summary_text']


# === Agent: Action Extractor ===
class ActionExtractorAgent:
    def __init__(self):
        self.actions = [
            'reset password', 'escalate to billing', 'update contact info',
            'follow up', 'network issue', 'installation error'
        ]

    def extract_actions(self, summary):
        actions_detected = [a for a in self.actions if a in summary.lower()]
        return actions_detected or ['manual review required']


# === Agent: Router ===
class RouterAgent:
    def __init__(self):
        self.routing_rules = {
            'reset password': 'Tech Team',
            'escalate to billing': 'Billing Team',
            'update contact info': 'Support Team',
            'follow up': 'Account Manager',
            'network issue': 'Network Ops',
            'installation error': 'Deployment Team',
            'manual review required': 'General Support'
        }

    def route(self, actions):
        return list(set(self.routing_rules.get(act, 'General Support') for act in actions))


# === Agent: Recommender using historical data ===
class RecommenderAgent:
    def __init__(self, ticket_df):
        self.df = ticket_df.copy()
        self.model = SentenceTransformer('all-MiniLM-L6-v2')

        # Infer column names
        issue_col = [col for col in self.df.columns if 'issue' in col.lower()][0]
        sentiment_col = [col for col in self.df.columns if 'sentiment' in col.lower()][0]
        solution_col = [col for col in self.df.columns if 'solution' in col.lower() or 'resolution' in col.lower()][0]

        self.issue_col = issue_col
        self.sentiment_col = sentiment_col
        self.solution_col = solution_col

        self.queries = (self.df[issue_col].astype(str) + ' ' + self.df[sentiment_col].astype(str)).tolist()
        self.embeddings = self.model.encode(self.queries, convert_to_tensor=True)

    def recommend_solution(self, issue_text):
        query_embedding = self.model.encode(issue_text, convert_to_tensor=True)
        scores = util.cos_sim(query_embedding, self.embeddings)[0]
        best_match_idx = int(scores.cpu().numpy().argmax())
        return self.df[self.solution_col].iloc[best_match_idx]


# === Agent: Estimator using ticket data ===
class EstimatorAgent:
    def __init__(self, ticket_df):
        self.df = ticket_df.copy()

        sentiment_col = [col for col in self.df.columns if 'sentiment' in col.lower()][0]
        priority_col = [col for col in self.df.columns if 'priority' in col.lower()][0]

        self.sentiment_col = sentiment_col
        self.priority_col = priority_col

        self.df['Resolution Days'] = 1  # Dummy value for training

        # Fit on known classes
        known_sentiments = ['Frustrated', 'Confused', 'Neutral']
        known_priorities = ['High', 'Medium', 'Low']

        self.le_sentiment = LabelEncoder()
        self.le_priority = LabelEncoder()
        self.le_sentiment.fit(known_sentiments)
        self.le_priority.fit(known_priorities)

        X = self.preprocess(self.df)
        y = self.df['Resolution Days']
        self.model = RandomForestRegressor().fit(X, y)

    def preprocess(self, df):
        sentiment_encoded = self.le_sentiment.transform(
            df[self.sentiment_col].apply(lambda x: x if x in self.le_sentiment.classes_ else 'Neutral')
        )
        priority_encoded = self.le_priority.transform(
            df[self.priority_col].apply(lambda x: x if x in self.le_priority.classes_ else 'Medium')
        )
        return np.column_stack([sentiment_encoded, priority_encoded])

    def estimate_time(self, sentiment, priority):
        sentiment = sentiment if sentiment in self.le_sentiment.classes_ else 'Neutral'
        priority = priority if priority in self.le_priority.classes_ else 'Medium'
        sentiment_code = self.le_sentiment.transform([sentiment])[0]
        priority_code = self.le_priority.transform([priority])[0]
        features = np.array([[sentiment_code, priority_code]])
        return round(float(self.model.predict(features)[0]), 2)


# === Coordinator Agent ===
class CoordinatorAgent:
    def __init__(self, ticket_df):
        self.summarizer = SummarizerAgent()
        self.extractor = ActionExtractorAgent()
        self.router = RouterAgent()
        self.recommender = RecommenderAgent(ticket_df)
        self.estimator = EstimatorAgent(ticket_df)

    def handle_query(self, user_input, sentiment='Frustrated', priority='High'):
        summary = self.summarizer.summarize(user_input)
        actions = self.extractor.extract_actions(summary)
        teams = self.router.route(actions)
        recommendation = self.recommender.recommend_solution(sentiment + " " + user_input)
        resolution_time = self.estimator.estimate_time(sentiment, priority)
        return summary, actions, teams, recommendation, resolution_time


# === Example usage without UI ===
if __name__ == '__main__':
    coordinator = CoordinatorAgent(df)

    user_input = input("Enter the customer issue: ")
    sentiment = input("Enter customer sentiment (Frustrated/Confused/Neutral): ")
    priority = input("Enter priority (High/Medium/Low): ")

    summary, actions, teams, recommendation, resolution_time = coordinator.handle_query(
        user_input, sentiment, priority
    )

    print("\n=== Summary ===")
    print(summary)
    print("\n=== Actions Extracted ===")
    print(actions)
    print("\n=== Routed Teams ===")
    print(teams)
    print("\n=== Recommended Resolution ===")
    print(recommendation)
    print("\n=== Estimated Resolution Time (hours) ===")
    print(resolution_time)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Enter the customer issue: Software Installation Failure
Enter customer sentiment (Frustrated/Confused/Neutral): Neutral
Enter priority (High/Medium/Low): High


Your max_length is set to 100, but your input_length is only 5. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=2)



=== Summary ===
Software Installation Failure. Software Installation Failure is a failure to install software on a computer. Software installation failure can occur when software is not installed on a PC or Mac.

=== Actions Extracted ===
['manual review required']

=== Routed Teams ===
['General Support']

=== Recommended Resolution ===
 Disable antivirus and retry installation

=== Estimated Resolution Time (hours) ===
1.0
